# Prueba 2

In [1]:
sc

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1643320542941_0001,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.
<SparkContext master=yarn appName=livy-session-0>

In [2]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext

df_yelp = spark.read.json('s3://bigdata-desafio/yelp-data/user.json')

### Ejercicio 1: Identificando usuarios molestosos (4.6 Puntos)
Utilizando el archivo user.json.
Desde Yelp están interesados en identificar a aquellos usuarios que se pueden considerar
como molestosos. Para ello, tienen la siguiente definición de un usuario molestoso:
- Un usuario molestoso es aquél que su promedio de evaluaciones es menor o igual a
2, tiene en promedio menos de 100 reviews y tiene cero fans.

A partir de esta definición, se le solicita los siguientes puntos:
- Identifique en una variable dummy todos los usuarios que se puedan clasificar como
molestosos acorde al criterio.
- Recodificaciones en el archivo user.json:
    - friends, que corresponde a un string con todos los user_id de otros
        usuarios j que siguen al usuario i. El objetivo es contar la cantidad de
        amigos existentes.
    - elite, que corresponde a un string con todos los años en los que el usuario
        i fue considerado como un reviewer de elite. El objetivo es contar la cantidad
        de años en los cuales se consideró como elite.
    - Asegúrese de eliminar los siguientes registros: friends, yelping_since,
        name, elite, user_id.


In [3]:
df_yelp.columns

['average_stars', 'compliment_cool', 'compliment_cute', 'compliment_funny', 'compliment_hot', 'compliment_list', 'compliment_more', 'compliment_note', 'compliment_photos', 'compliment_plain', 'compliment_profile', 'compliment_writer', 'cool', 'elite', 'fans', 'friends', 'funny', 'name', 'review_count', 'useful', 'user_id', 'yelping_since']

In [4]:
df_yelp.head(2)

[Row(average_stars=4.03, compliment_cool=1, compliment_cute=0, compliment_funny=1, compliment_hot=2, compliment_list=0, compliment_more=0, compliment_note=1, compliment_photos=0, compliment_plain=1, compliment_profile=0, compliment_writer=2, cool=25, elite=u'2015,2016,2017', fans=5, friends=u'c78V-rj8NQcQjOI8KP3UEA, alRMgPcngYSCJ5naFRBz5g, ajcnq75Z5xxkvUSmmJ1bCg, BSMAmp2-wMzCkhTfq9ToNg, jka10dk9ygX76hJG0gfPZQ, dut0e4xvme7QSlesOycHQA, l4l5lBnK356zBua7B-UJ6Q, 0HicMOOs-M_gl2eO-zES4Q, _uI57wL2fLyftrcSFpfSGQ, T4_Qd0YWbC3co6WSMw4vxg, iBRoLWPtWmsI1kdbE9ORSA, xjrUcid6Ymq0DoTJELkYyw, GqadWVzJ6At-vgLzK_SKgA, DvB13VJBmSnbFXBVBsKmDA, vRP9nQkYTeNioDjtxZlVhg, gT0A1iN3eeQ8EMAjJhwQtw, 6yCWjFPtp_AD4x93WAwmnw, 1dKzpNnib-JlViKv8_Gt5g, 3Bv4_JxHXq-gVLOxYMQX0Q, ikQyfu1iViYh8T0us7wiFQ, f1GGltNaB7K5DR1jf3dOmg, tgeFUChlh7v8bZFVl2-hjQ, -9-9oyXlqsMG2he5xIWdLQ, Adj9fBPVJad8vSs-mIP7gw, Ce49RY8CKXVsTifxRYFTsw, M1_7TLi8CbdA89nFLlH4iw, wFsNv-hqbW_F5-IRqfBN6g, 0Q1L7zXHocaUZ2gsG2XJeg, cBFgmOCBdhYa0xoFEAzp_g, VrD_AgiFvz

In [5]:
df_yelp.printSchema()

root
 |-- average_stars: double (nullable = true)
 |-- compliment_cool: long (nullable = true)
 |-- compliment_cute: long (nullable = true)
 |-- compliment_funny: long (nullable = true)
 |-- compliment_hot: long (nullable = true)
 |-- compliment_list: long (nullable = true)
 |-- compliment_more: long (nullable = true)
 |-- compliment_note: long (nullable = true)
 |-- compliment_photos: long (nullable = true)
 |-- compliment_plain: long (nullable = true)
 |-- compliment_profile: long (nullable = true)
 |-- compliment_writer: long (nullable = true)
 |-- cool: long (nullable = true)
 |-- elite: string (nullable = true)
 |-- fans: long (nullable = true)
 |-- friends: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- name: string (nullable = true)
 |-- review_count: long (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)
 |-- yelping_since: string (nullable = true)

In [10]:
from pyspark.sql.functions import when
# Se identifica la variable
df_yelp = df_yelp\
    .withColumn('molestoso', when((df_yelp['fans'] == 0) & (df_yelp['average_stars'] <= 2) & (df_yelp['review_count'] < 100), 1)\
    .otherwise(0))


In [27]:
# Generamos el DF final con el vector objetivo y las refactorizaciones de friends y elite.
df_yelp_final = df_yelp.rdd.map(lambda row: (row['average_stars'],
    row['compliment_cool'],
row['compliment_cute'],
row['compliment_funny'],
row['compliment_hot'],
row['compliment_list'],
row['compliment_more'],
row['compliment_note'],
row['compliment_photos'],
row['compliment_plain'],
row['compliment_profile'],
row['compliment_writer'],
row['cool'],
len(row['elite'].split(',')),
row['fans'],
len(row['friends'].split(',')),
row['funny'],
row['review_count'],
row['useful'],
row['molestoso']                             
)).toDF(['average_stars','compliment_cool',
'compliment_cute',
'compliment_funny',
'compliment_hot',
'compliment_list',
'compliment_more',
'compliment_note',
'compliment_photos',
'compliment_plain',
'compliment_profile',
'compliment_writer',
'cool',
'elite',
'fans',
'friends',
'funny',
'review_count',
'useful',
'molestoso'
])

In [28]:
# Medimos la cardinalidad según usuarios molestos
df_yelp_final.groupBy('molestoso').count().show()

+---------+-------+
|molestoso|  count|
+---------+-------+
|        0|1453462|
|        1| 183676|
+---------+-------+

In [29]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression, GBTClassifier, DecisionTreeClassifier


In [40]:
# Renombramos el vector objetivo y separamos el conjunto de atributos
df_yelp_final = df_yelp_final.withColumnRenamed('molestoso', 'label')
feats = df_yelp_final.columns
feats.remove('label')

In [41]:
vec = VectorAssembler(inputCols=feats, outputCol='assembled_features')



In [42]:
vec = vec.transform(df_yelp_final)

In [43]:
vec = vec.select('label', 'assembled_features')
vec.take(1)

[Row(label=0, assembled_features=DenseVector([4.03, 1.0, 0.0, 1.0, 2.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 2.0, 25.0, 3.0, 5.0, 99.0, 17.0, 95.0, 84.0]))]

In [45]:
train, test = vec.randomSplit([0.7, 0.3], seed=42)


In [49]:
# Modelo de regresión logística
logistic_example = LogisticRegression(featuresCol='assembled_features',
    labelCol='label',
    predictionCol='molestoso_pred')
logistic_example = logistic_example.fit(train)

Overall behavior
Area Under ROC: {model.areaUnderROC}
Accuracy: 0.999987780727
Behavior for class 0.0:
	Precision 0.999993117248
	Recall 0.999993117248
	True Positive Rate 0.999993117248
	False Positive Rate 5.43917448872e-05


Behavior for class 1.0:
	Precision 0.999945608255
	Recall 0.999945608255
	True Positive Rate 0.999945608255
	False Positive Rate 6.88275231433e-06

In [52]:
def report_metrics_on_pyspark_ml(model):
    print("Overall behavior")
    print("Area Under ROC: {}".format(model.areaUnderROC))
    print("Accuracy: {0}".format(model.accuracy))
    for index, values in enumerate(model.labels):
        print("Behavior for class {0}:".format(values))
        print("\tPrecision {}".format(model.precisionByLabel[index]))
        print("\tRecall {}".format(model.recallByLabel[index]))
        print("\tTrue Positive Rate {}".format(model.truePositiveRateByLabel[index]))
        print("\tFalse Positive Rate {}".format(model.falsePositiveRateByLabel[index]))
        print("\n")
report_metrics_on_pyspark_ml(logistic_example.evaluate(test))

Overall behavior
Area Under ROC: 0.999973973897
Accuracy: 0.999983720216
Behavior for class 0.0:
	Precision 0.99999541729
	Recall 0.999986251996
	True Positive Rate 0.999986251996
	False Positive Rate 3.63768643143e-05


Behavior for class 1.0:
	Precision 0.999890877346
	Recall 0.999963623136
	True Positive Rate 0.999963623136
	False Positive Rate 1.37480036753e-05

In [51]:
# Modelo de Gradient Boosting
gradient_model = GBTClassifier(featuresCol='assembled_features',
    labelCol='label',
    predictionCol='molestoso_pred').fit(train)
# Modelo Decision Tree
decision_tree_model = DecisionTreeClassifier(featuresCol='assembled_features',
    labelCol='label',
    predictionCol='molestoso_pred').fit(train)

In [55]:
gradient_test_df = gradient_model.transform(test)
gradient_test_df.take(2)
#report_metrics_on_pyspark_ml(decision_tree_model.evaluate(test))

[Row(label=0, assembled_features=SparseVector(19, {0: 3.79, 1: 1.0, 2: 1.0, 3: 1.0, 12: 4.0, 13: 1.0, 14: 2.0, 15: 49.0, 16: 12.0, 17: 13.0, 18: 32.0}), rawPrediction=DenseVector([1.5435, -1.5435]), probability=DenseVector([0.9564, 0.0436]), molestoso_pred=0.0), Row(label=0, assembled_features=SparseVector(19, {0: 4.6, 1: 1.0, 2: 1.0, 3: 1.0, 12: 2.0, 13: 1.0, 14: 2.0, 15: 79.0, 17: 10.0, 18: 7.0}), rawPrediction=DenseVector([1.5435, -1.5435]), probability=DenseVector([0.9564, 0.0436]), molestoso_pred=0.0)]

In [57]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator()
evaluator.setRawPredictionCol("molestoso_pred")


0.999974937566

In [60]:
models = {
    'Logistic': logistic_example,
    'Gradient': gradient_model,
    'DecisionTree': decision_tree_model
}
for name, model in models.items():
    print('AUC para {}: {}'.format(name, evaluator.evaluate(model.transform(test))))

AUC para Gradient: 0.999768575271
AUC para DecisionTree: 0.987283362263
AUC para Logistic: 0.999974937566

El mejor modelo es Logistic Regression.

In [63]:
for name, logodd in zip(feats, list(logistic_example.coefficients)):
    print(name, logodd)

('average_stars', -158.60348792129406)
('compliment_cool', 0.1687055049203947)
('compliment_cute', 0.9129427082992053)
('compliment_funny', 0.1687055049203947)
('compliment_hot', 0.1727011639970469)
('compliment_list', 0.8527244866188693)
('compliment_more', -0.128726537849921)
('compliment_note', -0.2429044846590442)
('compliment_photos', 0.0879267956359367)
('compliment_plain', 0.02576592048806627)
('compliment_profile', -0.5029559466304948)
('compliment_writer', -0.8044497778912548)
('cool', 0.06167836583574828)
('elite', -1861.0167728366898)
('fans', -189.00060230801432)
('friends', -0.0017960670271112784)
('funny', 0.037723462188272106)
('review_count', -0.16037107485530785)
('useful', -0.014423027103039216)

Los atributos que mas influyen son elite, fans, y average_stars, seguido finalmente por el único positivo compliment_cute